### try the VAF adjustment for one obvious sample

In [ ]:
import pandas as pd
import os
import sys
home = "/Users/martinscience"
home = "/Users/mahtin"
somvar_path = os.path.join(home, )
base_path = os.path.join(somvar_path, "Dropbox/Icke/Work/somVar/AMLMono7/WESData/AMLM74MarlonAdriane")

# get code
sys.path.append(os.path.join(home, "Sites/Bio/pytools"))
sys.path.append(os.path.join(home, "Sites/Bio/pycnv"))

from plot import make_CNV_plot
from working_with_mut_files import load_file, save_file, make_blocks, cat_chrom

SNV_path = os.path.join(base_path, "SNV")
CNV_path = os.path.join(base_path, "CNV")

### collect all the CNV_calls into one df

cnv_df = pd.read_csv(os.path.join(CNV_path, "CNV_collapsed.txt"), sep="\t")
cnv_df
cnv_df.loc[:, list(cnv_df.columns)[1:] + ['sample']].sort_values(['Chr', 'Start', 'End']).to_csv(os.path.join(CNV_path, "CNV_coll2.txt"), sep='\t', index=False)

### load collected CNV

In [ ]:
cnv_df = pd.read_csv(os.path.join(CNV_path, "CNV_coll3.txt"), sep="\t")
cnv_df = cat_chrom(cnv_df)
cnv_df['End'].max()

## collect the density of CNV per genomic stretch
+ first, get CNmajor/minor from cnacs-only calls

In [ ]:
# should this be flattened to 1?
cnv_df.loc[cnv_df['CNmajor'] == cnv_df['CNminor'], :]

## adjust cnv_df for display
+ imply CNmajor/minor from Kenichi cnacs
+ get coverage of CNV per gene stretch

In [ ]:
# assume LOH for TCN < 2
cnv_df.loc[(cnv_df['tool'] == "cnacs") & (cnv_df['TCN']< 2), ["CNmajor", "CNminor"]] = [1,0]
# assume CNmajor = |TCN| + 1
cnv_df.loc[(cnv_df['tool'] == "cnacs") & (cnv_df['TCN']> 2), ["CNmajor", "CNminor"]] = [cnv_df.loc[(cnv_df['tool'] == "cnacs") & (cnv_df['TCN']> 2), "TCN"].astype(int) + 1, 1]
# all cnacs has been implied
# cnv_df.loc[(cnv_df['tool'] == "cnacs"), :]
# select cols
df = cnv_df.loc[:, ['Chr', 'Start', 'End', 'CNmajor', 'CNminor', 'sample']]
# make step = 1 for start of CNV and -1 for end
df['step'] = 1

# merge start and "End"-df (End becomes step = -1)
flat_df = pd.concat( # concat the df and the df with End --> Start
    [df, df.drop(['Start', 'step'], axis=1).rename({'End':'Start'}, axis=1)]
).sort_values(['Chr', 'Start', 'End']).drop("End", axis=1).reset_index(drop=True)
# the step from the original End columns has to be negative
flat_df['step'] = flat_df['step'].fillna(-1).astype(int)
flat_df

### mark gains and losses
+ gains are defined as CNmajor + CNminor > 2
+ losses are defined as CNminor == 0
+ losses are displayed negative

In [ ]:
flat_df['gains'] = (flat_df['step'] * (flat_df['CNmajor'] + flat_df['CNminor'] > 2).astype(int)).cumsum().astype(int)
flat_df['losses'] = -(flat_df['step'] * (flat_df['CNminor'] == 0).astype(int)).cumsum().astype(int)
flat_df['cnv'] = flat_df['gains'] - flat_df['losses']
flat_df

### add full genomic positions for visualization

In [ ]:
chrom_df = load_file("/Users/mahtin/Dropbox/Icke/Work/static/genome/gatk/hg38/chroms.txt").iloc[:-2,:]
chrom_df['FullStart'] = chrom_df['len'].shift(1).cumsum().fillna(0).astype(int) + 1
chrom_df['FullEnd'] = chrom_df['len'].cumsum()
chrom_df

### add the cytoband

In [ ]:
cyto_df = pd.read_csv("/Users/mahtin/Dropbox/Icke/Work/static/annotation/annovar/humandb/hg38_cytoBand.txt", sep="\t", names=['Chr', 'Start', 'End', 'band', "strand"])
cyto_df = cyto_df.loc[cyto_df['Chr'].str.match("^chr[0-9]+$"), :]
cyto_df = cat_chrom(cyto_df).sort_values(['Chr', 'Start', 'End'])
cyto_df['BAND'] = cyto_df['band'].str[:1]
band_df = cyto_df.groupby(['Chr', 'BAND']).agg({'Start':'first', 'End':'last'}).dropna().reset_index()
for c in band_df.columns[2:]:
    band_df[c] = band_df[c].astype(int)
band_df[:10]

In [ ]:
fig_params = dict(
    figsize = (20,5),
    y_margin=4,
    label_size=20,
    y_label_size=15,
    chrom_label_yoffset = 5,
    chrom_color_scheme = "coolwarm_r",
    cnv_fill_colors={
        "gains":"#41c20e",
        "losses": "#ebc871"
    },
    chrom_label_size=1.25,
    band_label_yoffset=2,
    band_label_size=1.3,
    band_colors={
        "p":'none',
        "q":'lightgrey'
    }
)

fig, ax = make_CNV_plot(flat_df, chrom_df, band_df, **fig_params)

In [ ]:
fig_file = os.path.join("/Users/mahtin/Dropbox/Icke/Work/Abschlussbericht/figures", "CNVs.svg")

fig.savefig(fig_file)